In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#importing data (pre1790)
pre1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Pre1790.csv')
pre1790.columns = ['state', 'Loan Office Certificates Specie Value','State Liquidated Debt Dollars','Pierce Certificates Value']

In [3]:
#importing data (post1790)
post1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790.csv', index_col = 0)
post1790_ds = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_debtstate.csv', index_col = 0)
post1790_rep = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_rep.csv', index_col = 0)

In [4]:
state_pop = pd.read_csv('../Data/statepop.csv')[['State', 'Total Pop', 'Slave Pop']]

In [5]:
state_pop['White Pop'] = state_pop['Total Pop'] - state_pop['Slave Pop']
state_pop.dropna(inplace = True)
state_pop = state_pop[['State', 'White Pop']]
state_pop.columns = ['state', 'pop']

In [6]:
with open('state_abbrev.json') as file:
    state_abbrev = json.load(file)

In [7]:
state_pop['state'] = state_pop['state'].apply(lambda x: state_abbrev[x])

In [8]:
#pre 1790 and post1790 asset totals
pre1790cols = ['Loan Office Certificates Specie Value', 'State Liquidated Debt Dollars', 'Pierce Certificates Value']
post1790cols = ['ASD Total', 'CD Total']
pre1790['pre1790 total'] = pre1790[pre1790cols].sum(axis = 1)
post1790['post1790 total'] = post1790[post1790cols].sum(axis = 1)
post1790_ds['post1790 total'] = post1790_ds[post1790cols].sum(axis = 1)
post1790_rep['post1790 total'] = post1790_rep[post1790cols].sum(axis = 1)
pre1790_tot = pre1790['pre1790 total'].sum()
post1790_tot = post1790['post1790 total'].sum()
print("pre1790: {:.2f}, post1790: {:.2f}, ratio: {:.2f}".format(pre1790_tot, post1790_tot, post1790_tot/pre1790_tot))

pre1790: 26860611.73, post1790: 14476118.95, ratio: 0.54


In [9]:
def calculateMetrics(df):
    #aggregates
    df = df.fillna(0)
    df = pd.merge(df, state_pop, on = 'state', how = 'outer')
    
    df['pre1790 pct'] = df['pre1790 total']/pre1790_tot * 100
    df['post1790 pct'] = df['post1790 total']/post1790_tot * 100
    df['total asset dif'] = df['post1790 total'] - df['pre1790 total']
    #reweighting of post1790 assets
    df['total asset dif (adj)'] = df['post1790 total']*pre1790_tot/post1790_tot - all_debts_ds['pre1790 total']
    df['total asset pct dif'] = (df['post1790 pct'] - df['pre1790 pct'])
    
    df['Loan Office pct'] =  df['Loan Office Certificates Specie Value']/df['Loan Office Certificates Specie Value'].sum()
    df['State Liquidated pct'] =  df['State Liquidated Debt Dollars']/df['State Liquidated Debt Dollars'].sum()
    df['Pierce pct'] =  df['Pierce Certificates Value']/df['Pierce Certificates Value'].sum()
    df['CD pct'] =  df['CD Total']/df[ 'CD Total'].sum()
    df['ASD pct'] =  df['ASD Total']/df['ASD Total'].sum()
    
    df['pre1790 total (pop adj)'] = df['pre1790 total']/df['pop']
    df['post1790 total (pop adj)'] = df['post1790 total']/df['pop']
    
    df['pre1790 pct (pop adj)'] = df['pre1790 total (pop adj)']/df['pre1790 total (pop adj)'].sum() * 100
    df['post1790 pct (pop adj)'] = df['post1790 total (pop adj)']/df['post1790 total (pop adj)'].sum() * 100 
    df['total asset dif (pop adj)'] = df['post1790 total (pop adj)'] - df['pre1790 total (pop adj)']
    df['total asset pct dif (pop adj)'] =  df['post1790 pct (pop adj)'] - df['pre1790 pct (pop adj)']
    return df.round(2)

In [10]:
#default

In [11]:
all_debts = pd.merge(pre1790[['state', 'pre1790 total']], post1790[['state', 'post1790 total']], 
                     on = 'state', how = 'outer').round(2)
#all_debts

In [18]:
post1790_ds.columns = ['state', 'ASD Total', 'CD Total', 'post1790 total']
all_debts_ds = pd.merge(pre1790, post1790_ds, 
                        on = 'state', how = 'outer').round(2)
all_debts_ds = calculateMetrics(all_debts_ds)
#all_debts_ds

In [13]:
post1790_rep.loc[post1790_rep[post1790_rep['state'] == 'NY loan office'].index, 'state'] = 'NY'

In [17]:
post1790_rep.loc[post1790_rep[post1790_rep['state'] == 'NY loan office'].index, 'state'] = 'NY'
post1790_rep = post1790_rep.groupby('state')[['ASD Total','CD Total', 'post1790 total']].sum().reset_index()
all_debts_rep = pd.merge(pre1790, post1790_rep, 
                        on = 'state', how = 'outer').round(2)
all_debts_rep = calculateMetrics(all_debts_rep)
#all_debts_rep

In [15]:
all_debts_ds.to_csv('statewise_all_assets_ds.csv')

In [16]:
all_debts_rep.to_csv('statewise_all_assets_rep.csv')